# Incremental Learning 1: Two-Step Incremental Learning

1. Train model (M1) on the complete dataset (D1).
2. Split the complete dataset (D1) into the base dataset (D2) and the update dataset (D3).
3. Train a new model (M2) on D2 and update it using D3.
4. Compare M1 and M2
    1. Evaluation M1 and M2 on the complete dataset D1.
    2. TODO: Compare feature importance vectors for individual data points from M1 and M2 (should be very similar)


In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import max_error, mean_absolute_error, mean_squared_error

from incremental_learning.misc import train, summarize, update, evaluate
from incremental_learning.config import datasets_dir, root_dir

## 1. Train model (M1) on the complete dataset (D1)

In [2]:
dataset_name = 'ccpp'
D1 = pd.read_csv(datasets_dir / '{}.csv'.format(dataset_name))
D1.drop_duplicates(inplace=True)

## sample 1000 rows just for test purposes!
D1 = D1.sample(100)

job1 = train(dataset_name, D1)

session: job_kotbi	command:
/home/valeriy/Documents/workspace/valeriy42/ml-cpp/build/distribution/platform/linux-x86_64/bin/data_frame_analyzer --input /tmp/tmpararuyer --config /tmp/tmpha2frn1_ --output /tmp/tmpkft_ma56 --persist /tmp/tmpir83vcn1; if [ $? -eq 0 ]; then echo "Success"; else echo "Failure";  fi;


In [3]:
job1.wait_to_complete()

stderr,output
"/home/valeriy/Documents/workspace/valeriy42/ml-cpp/build/distribution/platform/linux-x86_64/bin/data_frame_analyzer --input /tmp/tmpararuyer --config /tmp/tmpha2frn1_ --output /tmp/tmpkft_ma56 --persist /tmp/tmpir83vcn1; if [ $? -eq 0 ]; then echo ""Success""; else echo ""Failure""; fi (env) ✔ ~/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/incremental_learning [python-tests|✚ 3…1⚑ 6] 13:20 $ /home/valeriy/Documents/workspace/valeriy42/ml-cpp/build/distribution/platform/linux-x86_64/bin/data_frame_analyzer --input /tmp/tmpararuyer --config /tmp/tmpha2frn1_ --output /tmp/tmpkft_ma56 --persist /tmp/tmpir83vcn1; if [ $? -eq 0 ]; then echo ""Success""; else echo ""Failure""; fi 2021-06-30 11:20:36,570727 UTC [20984] DEBUG Main.cc@145 data_frame_analyzer (64 bit): Version based on 8.0.0-SNAPSHOT (Build DEVELOPMENT BUILD by valeriy) Copyright (c) 2021 Elasticsearch BV 2021-06-30 11:20:36,570894 UTC [20984] DEBUG CProcessPriority_Linux.cc@33 Successfully increased OOM killer adjustment via /proc/self/oom_score_adj 2021-06-30 11:20:36,570934 UTC [20984] DEBUG CSystemCallFilter_Linux.cc@128 Seccomp BPF filters available 2021-06-30 11:20:36,574516 UTC [20984] DEBUG CSystemCallFilter_Linux.cc@154 Seccomp BPF installed 2021-06-30 11:20:36,576074 UTC [20984] DEBUG CDataFrameAnalyzer.cc@102 Received 100 rows 2021-06-30 11:21:06,762045 UTC [20984] INFO CBoostedTreeImpl.cc@260 Exiting hyperparameter optimisation loop early 2021-06-30 11:21:07,318350 UTC [20984] INFO Main.cc@248 [{""name"":""E_DFTPMEstimatedPeakMemoryUsage"",""description"":""The upfront estimate of the peak memory training the predictive model would use"",""value"":4189860} ,{""name"":""E_DFTPMPeakMemoryUsage"",""description"":""The peak memory training the predictive model used"",""value"":400087} ,{""name"":""E_DFTPMTimeToTrain"",""description"":""The time it took to train the predictive model"",""value"":30540} ,{""name"":""E_DFTPMTrainedForestNumberTrees"",""description"":""The total number of trees in the trained forest"",""value"":64} ] 2021-06-30 11:21:07,318480 UTC [20984] DEBUG Main.cc@253 ML data frame analyzer exiting Success (env) ✔ ~/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/incremental_learning [python-tests|✚ 3…1⚑ 6] 13:21 $",",{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":441.5649108886719 ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":440.3795166015625 ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":444.5744323730469 ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":467.915283203125, ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":465.50439453125,"" ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":475.690185546875, ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":435.4629211425781 ,{""model_metadata"":{""total_feature_importance"":[],""hyperparameters"":[{""name"":""do ,{""compressed_data_summarization"":{""doc_num"":0,""data_summarization"":""H4sIAAAAAAA ]"


 /home/valeriy/Documents/workspace/valeriy42/ml-cpp/build/distribution/platform/linux-x86_64/bin/data_frame_analyzer --input /tmp/tmpararuyer --config /tmp/tmpha2frn1_ --output /tmp/tmpkft_ma56 --persist /tmp/tmpir83vcn1; if [ $? -eq 0 ]; then echo "Success"; else echo "Failure";  fi
(env) ✔ ~/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/incremental_learning [python-tests|✚ 3…1⚑ 6]
13:20 $  /home/valeriy/Documents/workspace/valeriy42/ml-cpp/build/distribution/platform/linux-x86_64/bin/data_frame_analyzer --input /tmp/tmpararuyer --config /tmp/tmpha2frn1_ --output /tmp/tmpkft_ma56 --persist /tmp/tmpir83vcn1; if [ $? -eq 0 ]; then echo "Success"; else echo "Failure";  fi
2021-06-30 11:20:36,570727 UTC [20984] DEBUG Main.cc@145 data_frame_analyzer (64 bit): Version based on 8.0.0-SNAPSHOT (Build DEVELOPMENT BUILD by valeriy) Copyright (c) 2021 Elasticsearch BV
2021-06-30 11:20:36,570894 UTC [20984] DEBUG CProcessPriority_Linux.cc@33 Successfully increased OOM killer adjustment v

True

## 2. Split the complete dataset (D1) into the base dataset (D2) and the update dataset (D3).

In [4]:
D2, D3 = train_test_split(D1, test_size=0.2)

## 3. Train a new model (M2) on D2 and update it using D3.

In [5]:
job2 = train(dataset_name, D2)

session: job_sglmf	command:
/home/valeriy/Documents/workspace/valeriy42/ml-cpp/build/distribution/platform/linux-x86_64/bin/data_frame_analyzer --input /tmp/tmprtyvrvwn --config /tmp/tmpzpp0v0da --output /tmp/tmp7224rcfl --persist /tmp/tmpju9y4ln6; if [ $? -eq 0 ]; then echo "Success"; else echo "Failure";  fi;


In [6]:
job2.wait_to_complete()

stderr,output
"/home/valeriy/Documents/workspace/valeriy42/ml-cpp/build/distribution/platform/linux-x86_64/bin/data_frame_analyzer --input /tmp/tmprtyvrvwn --config /tmp/tmpzpp0v0da --output /tmp/tmp7224rcfl --persist /tmp/tmpju9y4ln6; if [ $? -eq 0 ]; then echo ""Success""; else echo ""Failure""; fi (env) ✔ ~/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/incremental_learning [python-tests|✚ 3…1⚑ 6] 13:21 $ /home/valeriy/Documents/workspace/valeriy42/ml-cpp/build/distribution/platform/linux-x86_64/bin/data_frame_analyzer --input /tmp/tmprtyvrvwn --config /tmp/tmpzpp0v0da --output /tmp/tmp7224rcfl --persist /tmp/tmpju9y4ln6; if [ $? -eq 0 ]; then echo ""Success""; else echo ""Failure""; fi 2021-06-30 11:21:12,711768 UTC [24445] DEBUG Main.cc@145 data_frame_analyzer (64 bit): Version based on 8.0.0-SNAPSHOT (Build DEVELOPMENT BUILD by valeriy) Copyright (c) 2021 Elasticsearch BV 2021-06-30 11:21:12,711829 UTC [24445] DEBUG CProcessPriority_Linux.cc@33 Successfully increased OOM killer adjustment via /proc/self/oom_score_adj 2021-06-30 11:21:12,711840 UTC [24445] DEBUG CSystemCallFilter_Linux.cc@128 Seccomp BPF filters available 2021-06-30 11:21:12,712897 UTC [24445] DEBUG CSystemCallFilter_Linux.cc@154 Seccomp BPF installed 2021-06-30 11:21:12,715445 UTC [24445] DEBUG CDataFrameAnalyzer.cc@102 Received 80 rows 2021-06-30 11:21:37,346464 UTC [24445] INFO CBoostedTreeImpl.cc@260 Exiting hyperparameter optimisation loop early 2021-06-30 11:21:38,363548 UTC [24445] INFO Main.cc@248 [{""name"":""E_DFTPMEstimatedPeakMemoryUsage"",""description"":""The upfront estimate of the peak memory training the predictive model would use"",""value"":3496286} ,{""name"":""E_DFTPMPeakMemoryUsage"",""description"":""The peak memory training the predictive model used"",""value"":281909} ,{""name"":""E_DFTPMTimeToTrain"",""description"":""The time it took to train the predictive model"",""value"":24980} ,{""name"":""E_DFTPMTrainedForestNumberTrees"",""description"":""The total number of trees in the trained forest"",""value"":127} ] 2021-06-30 11:21:38,363670 UTC [24445] DEBUG Main.cc@253 ML data frame analyzer exiting Success (env) ✔ ~/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/incremental_learning [python-tests|✚ 3…1⚑ 6] 13:21 $",",{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":436.8063354492187 ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":469.8267517089844 ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":478.8609313964844 ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":444.1383361816406 ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":439.8424377441406 ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":448.0448913574219 ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":441.6157531738281 ,{""model_metadata"":{""total_feature_importance"":[],""hyperparameters"":[{""name"":""do ,{""compressed_data_summarization"":{""doc_num"":0,""data_summarization"":""H4sIAAAAAAA ]"


 /home/valeriy/Documents/workspace/valeriy42/ml-cpp/build/distribution/platform/linux-x86_64/bin/data_frame_analyzer --input /tmp/tmprtyvrvwn --config /tmp/tmpzpp0v0da --output /tmp/tmp7224rcfl --persist /tmp/tmpju9y4ln6; if [ $? -eq 0 ]; then echo "Success"; else echo "Failure";  fi
(env) ✔ ~/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/incremental_learning [python-tests|✚ 3…1⚑ 6]
13:21 $  /home/valeriy/Documents/workspace/valeriy42/ml-cpp/build/distribution/platform/linux-x86_64/bin/data_frame_analyzer --input /tmp/tmprtyvrvwn --config /tmp/tmpzpp0v0da --output /tmp/tmp7224rcfl --persist /tmp/tmpju9y4ln6; if [ $? -eq 0 ]; then echo "Success"; else echo "Failure";  fi
2021-06-30 11:21:12,711768 UTC [24445] DEBUG Main.cc@145 data_frame_analyzer (64 bit): Version based on 8.0.0-SNAPSHOT (Build DEVELOPMENT BUILD by valeriy) Copyright (c) 2021 Elasticsearch BV
2021-06-30 11:21:12,711829 UTC [24445] DEBUG CProcessPriority_Linux.cc@33 Successfully increased OOM killer adjustment v

True

In [7]:
job3 = update(dataset_name, D3, job2)
job3.wait_to_complete()

stderr,output
"/home/valeriy/Documents/workspace/valeriy42/ml-cpp/build/distribution/platform/linux-x86_64/bin/data_frame_analyzer --input /tmp/tmpuvkru86s --config /tmp/tmpaaj_2jv0 --output /tmp/tmp9ze5jjdp --restore /tmp/tmp3cvi4gjv; if [ $? -eq 0 ]; then echo ""Success""; else echo ""Failure""; fi (env) ✔ ~/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/incremental_learning [python-tests|✚ 3…1⚑ 6] 13:21 $ /home/valeriy/Documents/workspace/valeriy42/ml-cpp/build/distribution/platform/linux-x86_64/bin/data_frame_analyzer --input /tmp/tmpuvkru86s --config /tmp/tmpaaj_2jv0 --output /tmp/tmp9ze5jjdp --restore /tmp/tmp3cvi4gjv; if [ $? -eq 0 ]; then echo ""Success""; else echo ""Failure""; fi 2021-06-30 11:21:43,622556 UTC [27421] DEBUG Main.cc@145 data_frame_analyzer (64 bit): Version based on 8.0.0-SNAPSHOT (Build DEVELOPMENT BUILD by valeriy) Copyright (c) 2021 Elasticsearch BV 2021-06-30 11:21:43,622780 UTC [27421] DEBUG CProcessPriority_Linux.cc@33 Successfully increased OOM killer adjustment via /proc/self/oom_score_adj 2021-06-30 11:21:43,622848 UTC [27421] DEBUG CSystemCallFilter_Linux.cc@128 Seccomp BPF filters available 2021-06-30 11:21:43,623750 UTC [27421] DEBUG CSystemCallFilter_Linux.cc@154 Seccomp BPF installed 2021-06-30 11:21:43,636307 UTC [27421] DEBUG CDataFrameAnalyzer.cc@102 Received 28 rows 2021-06-30 11:21:43,681464 UTC [27421] INFO Main.cc@248 [{""name"":""E_DFTPMEstimatedPeakMemoryUsage"",""description"":""The upfront estimate of the peak memory training the predictive model would use"",""value"":1205103} ,{""name"":""E_DFTPMPeakMemoryUsage"",""description"":""The peak memory training the predictive model used"",""value"":72563} ,{""name"":""E_DFTPMTimeToTrain"",""description"":""The time it took to train the predictive model"",""value"":16} ] 2021-06-30 11:21:43,681532 UTC [27421] DEBUG Main.cc@253 ML data frame analyzer exiting Success (env) ✔ ~/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/incremental_learning [python-tests|✚ 3…1⚑ 6] 13:21 $",",{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":433.8936462402344 ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":457.052490234375, ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":449.6717834472656 ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":443.0817565917969 ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":469.8158264160156 ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":461.143310546875, ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":431.4468383789062 ,{""model_metadata"":{""total_feature_importance"":[],""hyperparameters"":[{""name"":""do ,{""compressed_data_summarization"":{""doc_num"":0,""data_summarization"":""H4sIAAAAAAA ]"


 /home/valeriy/Documents/workspace/valeriy42/ml-cpp/build/distribution/platform/linux-x86_64/bin/data_frame_analyzer --input /tmp/tmpuvkru86s --config /tmp/tmpaaj_2jv0 --output /tmp/tmp9ze5jjdp --restore /tmp/tmp3cvi4gjv; if [ $? -eq 0 ]; then echo "Success"; else echo "Failure";  fi
(env) ✔ ~/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/incremental_learning [python-tests|✚ 3…1⚑ 6]
13:21 $  /home/valeriy/Documents/workspace/valeriy42/ml-cpp/build/distribution/platform/linux-x86_64/bin/data_frame_analyzer --input /tmp/tmpuvkru86s --config /tmp/tmpaaj_2jv0 --output /tmp/tmp9ze5jjdp --restore /tmp/tmp3cvi4gjv; if [ $? -eq 0 ]; then echo "Success"; else echo "Failure";  fi
2021-06-30 11:21:43,622556 UTC [27421] DEBUG Main.cc@145 data_frame_analyzer (64 bit): Version based on 8.0.0-SNAPSHOT (Build DEVELOPMENT BUILD by valeriy) Copyright (c) 2021 Elasticsearch BV
2021-06-30 11:21:43,622780 UTC [27421] DEBUG CProcessPriority_Linux.cc@33 Successfully increased OOM killer adjustment v

True

## 4. A. Compare M1 and M2 on D1

In [7]:
y_true = D1[job1.dependent_variable]
y_M1 = job1.get_predictions()
eval_job = evaluate(dataset_name, D1, job2)
success = eval_job.wait_to_complete()
if not success:
    print('Evaluation failed')
y_M2 = eval_job.get_predictions()

stderr,output
"/home/valeriy/Documents/workspace/valeriy42/ml-cpp/build/distribution/platform/linux-x86_64/bin/data_frame_analyzer --input /tmp/tmpb6r09iv4 --config /tmp/tmp7qr5e_co --output /tmp/tmp90gtakrf --restore /tmp/tmpa4d1xn7u; if [ $? -eq 0 ]; then echo ""Success""; else echo ""Failure""; fi (env) ✔ ~/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/incremental_learning [python-tests|✚ 3…1⚑ 6] 11:24 $ /home/valeriy/Documents/workspace/valeriy42/ml-cpp/build/distribution/platform/linux-x86_64/bin/data_frame_analyzer --input /tmp/tmpb6r09iv4 --config /tmp/tmp7qr5e_co --output /tmp/tmp90gtakrf --restore /tmp/tmpa4d1xn7u; if [ $? -eq 0 ]; then echo ""Success""; else echo ""Failure""; fi 2021-06-30 09:24:22,141137 UTC [3034] DEBUG Main.cc@145 data_frame_analyzer (64 bit): Version based on 8.0.0-SNAPSHOT (Build DEVELOPMENT BUILD by valeriy) Copyright (c) 2021 Elasticsearch BV 2021-06-30 09:24:22,141186 UTC [3034] DEBUG CProcessPriority_Linux.cc@33 Successfully increased OOM killer adjustment via /proc/self/oom_score_adj 2021-06-30 09:24:22,141197 UTC [3034] DEBUG CSystemCallFilter_Linux.cc@128 Seccomp BPF filters available 2021-06-30 09:24:22,141616 UTC [3034] DEBUG CSystemCallFilter_Linux.cc@154 Seccomp BPF installed 2021-06-30 09:24:22,151253 UTC [3034] DEBUG CDataFrameAnalyzer.cc@102 Received 103 rows 2021-06-30 09:24:22,175340 UTC [3034] INFO Main.cc@248 [{""name"":""E_DFTPMEstimatedPeakMemoryUsage"",""description"":""The upfront estimate of the peak memory training the predictive model would use"",""value"":4273143} ,{""name"":""E_DFTPMPeakMemoryUsage"",""description"":""The peak memory training the predictive model used"",""value"":80958} ,{""name"":""E_DFTPMTimeToTrain"",""description"":""The time it took to train the predictive model"",""value"":8} ] 2021-06-30 09:24:22,175375 UTC [3034] DEBUG Main.cc@253 ML data frame analyzer exiting Success (env) ✔ ~/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/incremental_learning [python-tests|✚ 3…1⚑ 6] 11:24 $",",{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":464.59326171875,"" ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":438.0891418457031 ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":465.5749816894531 ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":434.2337341308594 ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":445.0762939453125 ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":482.8708190917969 ,{""row_results"":{""checksum"":0,""results"":{""ml"":{""PE_prediction"":439.6938171386719 ,{""model_metadata"":{""total_feature_importance"":[],""hyperparameters"":[],""data_sum ,{""compressed_data_summarization"":{""doc_num"":0,""data_summarization"":""H4sIAAAAAAA ]"


 /home/valeriy/Documents/workspace/valeriy42/ml-cpp/build/distribution/platform/linux-x86_64/bin/data_frame_analyzer --input /tmp/tmpb6r09iv4 --config /tmp/tmp7qr5e_co --output /tmp/tmp90gtakrf --restore /tmp/tmpa4d1xn7u; if [ $? -eq 0 ]; then echo "Success"; else echo "Failure";  fi
(env) ✔ ~/Documents/workspace/valeriy42/ml-cpp/jupyter/notebooks/incremental_learning [python-tests|✚ 3…1⚑ 6]
11:24 $  /home/valeriy/Documents/workspace/valeriy42/ml-cpp/build/distribution/platform/linux-x86_64/bin/data_frame_analyzer --input /tmp/tmpb6r09iv4 --config /tmp/tmp7qr5e_co --output /tmp/tmp90gtakrf --restore /tmp/tmpa4d1xn7u; if [ $? -eq 0 ]; then echo "Success"; else echo "Failure";  fi
2021-06-30 09:24:22,141137 UTC [3034] DEBUG Main.cc@145 data_frame_analyzer (64 bit): Version based on 8.0.0-SNAPSHOT (Build DEVELOPMENT BUILD by valeriy) Copyright (c) 2021 Elasticsearch BV
2021-06-30 09:24:22,141186 UTC [3034] DEBUG CProcessPriority_Linux.cc@33 Successfully increased OOM killer adjustment via

In [8]:
def compute_metrics(ytrue, m1pred, m2pred):
    m1_mae = mean_absolute_error(ytrue, m1pred)
    m1_mse = mean_squared_error(ytrue, m1pred)
    m2_mae = mean_absolute_error(ytrue, m2pred)
    m2_mse = mean_squared_error(ytrue, m2pred)
    print("M1: MAE: {}\tMSE:{}".format(m1_mae, m1_mse))
    print("M2: MAE: {}\tMSE:{}".format(m2_mae, m2_mse))
    ax = sns.scatterplot(x=m1pred, y=m2pred)
    plt.xlabel('M1 predictions')
    plt.ylabel('M2 predictions')

In [9]:
compute_metrics(y_true, y_M1, y_M2)

ValueError: Found input variables with inconsistent numbers of samples: [100, 103]